***GENERATED CODE FOR amazonfinal PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Date0', 'transformation_label': 'String Indexer'}], 'feature': 'Date0', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '05/02/2019', 'max': '05/02/2019', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Date0'}, {'feature_label': 'Date0', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Date0')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Date3', 'transformation_label': 'String Indexer'}], 'feature': 'Date3', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '05/02/2019', 'max': '05/02/2019', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Date3'}, {'feature_label': 'Date3', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Date3')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Weekly_Sales', 'threshold': 14505.904, 'transformation_label': 'Binarizer'}], 'feature': 'Weekly_Sales', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '14506.35', 'stddev': '15963.78', 'min': '0.0', 'max': '70677.59', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'Weekly_Sales'}, {'feature_label': 'Weekly_Sales', 'threshold': 14505.904, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('Weekly_Sales')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Type', 'transformation_label': 'String Indexer'}], 'feature': 'Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'C', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Type'}, {'feature_label': 'Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Type')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'max', 'threshold': 34071.176, 'transformation_label': 'Binarizer'}], 'feature': 'max', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '34071.67', 'stddev': '42262.48', 'min': '21.96', 'max': '406988.63', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'max'}, {'feature_label': 'max', 'threshold': 34071.176, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('max')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'mean', 'threshold': 14216.916, 'transformation_label': 'Binarizer'}], 'feature': 'mean', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '14217.42', 'stddev': '15190.97', 'min': '5.411803279', 'max': '75522.87441', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'mean'}, {'feature_label': 'mean', 'threshold': 14216.916, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('mean')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'median', 'threshold': 13205.72, 'transformation_label': 'Binarizer'}], 'feature': 'median', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '13206.18', 'stddev': '14741.74', 'min': '2.66', 'max': '75506.54', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'median'}, {'feature_label': 'median', 'threshold': 13205.72, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('median')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run amazonfinalHooks.ipynb
try:
	#sourcePreExecutionHook()

	amazonf = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/AmazonScaledVersion.csv', 'filename': 'AmazonScaledVersion.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run amazonfinalHooks.ipynb
try:
	#sourcePreExecutionHook()

	amazonscaledversion = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/AmazonScaledVersion.csv', 'filename': 'AmazonScaledVersion.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(amazonscaledversion)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run amazonfinalHooks.ipynb
try:
	#transformationPreExecutionHook()

	amazonfinalautofe = TransformationMain.run(amazonscaledversion,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Date0", "transformation_label": "String Indexer"}], "feature": "Date0", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "05/02/2019", "max": "05/02/2019", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Date0"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "10.93", "stddev": "5.48", "min": "1", "max": "42", "missing": "0"}, "updatedLabel": "Store"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Dept", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "42.2", "stddev": "29.32", "min": "1", "max": "98", "missing": "0"}, "updatedLabel": "Dept"}, {"transformationsData": [{"feature_label": "Date3", "transformation_label": "String Indexer"}], "feature": "Date3", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "05/02/2019", "max": "05/02/2019", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Date3"}, {"transformationsData": [{"feature_label": "Weekly_Sales", "threshold": 14505.904, "transformation_label": "Binarizer"}], "feature": "Weekly_Sales", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "14506.35", "stddev": "15963.78", "min": "0.0", "max": "70677.59", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "Weekly_Sales"}, {"transformationsData": [{"feature_label": "Type", "transformation_label": "String Indexer"}], "feature": "Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "C", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Size", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "160313.26", "stddev": "55617.04", "min": "34875", "max": "219622", "missing": "0"}, "updatedLabel": "Size"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Temperature", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "36.15", "stddev": "9.5", "min": "20.96", "max": "54.34", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Temperature"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Fuel_Price", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.7", "stddev": "0.15", "min": "2.572", "max": "2.962", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Fuel_Price"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CPI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "179.97", "stddev": "40.17", "min": "126.4420645", "max": "214.6554591", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CPI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Unemployment", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.75", "stddev": "1.16", "min": "6.299", "max": "9.765", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Unemployment"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsHoliday", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "IsHoliday"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2019.0", "stddev": "0.0", "min": "2019", "max": "2019", "missing": "0"}, "updatedLabel": "Year"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.0", "stddev": "0.0", "min": "2", "max": "2", "missing": "0"}, "updatedLabel": "Month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Week", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "5.0", "stddev": "0.0", "min": "5", "max": "5", "missing": "0"}, "updatedLabel": "Week"}, {"transformationsData": [{"feature_label": "max", "threshold": 34071.176, "transformation_label": "Binarizer"}], "feature": "max", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "34071.67", "stddev": "42262.48", "min": "21.96", "max": "406988.63", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "max"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "min", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "8773.33", "stddev": "11051.24", "min": "-370.12", "max": "52631.53", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "min"}, {"transformationsData": [{"feature_label": "mean", "threshold": 14216.916, "transformation_label": "Binarizer"}], "feature": "mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "14217.42", "stddev": "15190.97", "min": "5.411803279", "max": "75522.87441", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "mean"}, {"transformationsData": [{"feature_label": "median", "threshold": 13205.72, "transformation_label": "Binarizer"}], "feature": "median", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "13206.18", "stddev": "14741.74", "min": "2.66", "max": "75506.54", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "median"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "std", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "4210.54", "stddev": "5806.44", "min": "6.674093573", "max": "49692.18557", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "std"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total_MarkDown", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Total_MarkDown"}]}))

	#transformationPostExecutionHook(amazonfinalautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run amazonfinalHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(amazonfinalautofe, ["Store", "Dept", "Size", "Temperature", "Fuel_Price", "Unemployment", "IsHoliday", "Year", "Month", "Week", "min", "std", "Total_MarkDown", "Date0_stringindexer", "Date3_stringindexer", "Weekly_Sales_binarizer", "Type_stringindexer", "max_binarizer", "mean_binarizer", "median_binarizer"], "CPI")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

